## 휴먼지능정보공학과 201910787 김영서

'제74주년 경찰의 날 기념식 축사' 전문을 읽고 TF-IDF를 계산하세요.

전문은 http://www.korea.kr/archive/speechView.do?newsId=132031636 에서 읽을 수 있고,

해당 사이트에서 텍스트만 파일로 저장한 후 사용한다.



* 1) DataFrame을 생성

* 2) 단어로 분리해서, 출력

* 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

* 4) 불용어 제거하고, 출력

* 5) TF-IDF를 계산하고, 출력

* 6) TF-IDF 컬럼을 features로 구성, 출력



In [1]:
import sys
import os

home=os.path.expanduser("~")
os.environ["PYSPARK_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.9-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [3]:
%%writefile data/police.txt
존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.

국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.

오늘 홍조근정훈장을 받으신 중앙경찰학교장 이은정 치안감님, 근정포장을 받으신 광주남부경찰서 김동현 경감님을 비롯한 수상자 여러분께 각별한 축하와 감사를 드립니다. 또한 경찰 영웅으로 추서되신 차일혁, 최중락님께 국민의 사랑을 전해드립니다.

사랑하는 경찰관 여러분,

여러분의 헌신적 노력으로 우리의 치안은 참 좋아졌습니다. 지난해 범죄 발생은 2015년에 비해 15.1% 줄었습니다. 같은 기간 교통사고 사망자는 18.2% 감소했습니다.

치안의 개선은 국민의 체감으로 나타나고 있습니다. 올해 상반기 국민의 체감안전도는 74.5점으로 역대 최고를 기록했습니다. 범죄안전도는 처음으로 80점을 넘었습니다.

한국을 찾는 외국 관광객들도 우리의 치안을 가장 좋게 평가합니다. 한국의 무엇이 좋았느냐는 물음에 외국 관광객들은 7년 연속으로 치안이 가장 좋았다고 응답했습니다. 개발도상국들은 우리의 경찰을 모범으로 삼으려 합니다.

올해는 ‘경찰의 날’에 맞춰 국제치안산업박람회와 서울국제경찰청장회의가 함께 열립니다. 우리의 치안 발전과 치안산업 발전이 세계에 더 널리 알려지게 될 것입니다.

자랑스러운 경찰관 여러분,

경찰헌장은 “나라와 겨레를 위하여 충성”을 다한다는 다짐으로 시작합니다. 헌장처럼 우리 경찰은 ‘나라와 겨레를 위한 충성’의 길을 걸으려 노력해 왔습니다.

대한민국 경찰은 1945년 광복 직후에 공식 탄생했습니다. 그러나 그 뿌리는 대한민국 임시정부에 닿아 있습니다.

임시정부 초대 경무국장 백범 김구 선생과 나석주, 나창헌, 유상근 의사 등 임시정부 경찰은 앞장서서 일제와 싸웠습니다. 일본 관헌에게 폭탄을 던지고, 밀정을 응징하며, 임정 요인들을 보호했습니다.

광복 이후 6·25전쟁에서도 경찰은 국군과 함께 피를 흘렸습니다. 그 전쟁에서 1만여 명의 경찰관이 목숨을 잃었습니다. 그 후로도 경찰은 국민의 안전을 지키고 국가의 안보를 도왔습니다. 역대 경찰의 헌신에 대해 국민과 함께 거듭 감사의 말씀을 드립니다. 감사합니다.

그러나 잘못도 없지는 않았습니다. 한때 경찰은 공권력을 무리하게 집행하며 국민의 인권을 훼손했습니다. 부실하거나 불공정한 수사로 국민의 지탄을 받은 적도 있습니다. 무기력한 법 집행으로 국민께 걱정을 드리기도 했습니다.

지금 경찰은 과거를 돌아보며 국민과 국가에 충성하는 경찰로 거듭나려고 노력하고 있습니다. 경찰은 문재인 정부 들어 가장 먼저 개혁위원회를 만들고 자체개혁에 나섰습니다. 경찰의 개혁을 국민은 큰 기대로 주목하고 있습니다.

검경 수사권 조정과 자치경찰제 도입이 국회에서 논의되고 있습니다. 국회가 조속히 입법을 매듭지어 주시기 바랍니다. 그리하여 경찰이 중립성, 공정성, 전문성을 갖추고 본연의 임무를 충실히 수행하는 선진경찰로 더욱 발전해 주기를 소망합니다.

정부는 경찰의 근무여건을 개선하기 위해 노력하고 있습니다. 이미 경찰관 8,572명을 늘렸고, 앞으로도 충원을 계속할 것입니다. 특히 일선 경찰의 근무환경을 개선하겠습니다.

정부는 누구도 법 위에 군림하지 못하는 법치주의를 확립하고자 합니다. 그러자면 검찰과 경찰이 법을 누구에게나 엄정하고 공정하게 집행해야 됩니다. 수사 또한 엄정하고 공정해야 합니다. 동시에 검찰과 경찰 스스로도 법을 엄격히 준수해야 합니다. 특히 공권력이 인권의 제약을 수반하는 경우에는 절제하며 행사하는 것이 마땅합니다. 검찰개혁과 경찰개혁은 더 미룰 수 없는 시대적 과제가 됐습니다.

경찰헌장은 따뜻한 경찰, 의로운 경찰, 공정한 경찰을 다짐합니다. 흔들림 없이 그 길로 가시기 바랍니다. 국민이 여러분을 응원할 것입니다.

행사를 준비하신 민갑룡 경찰청장님과 관계자 여러분, 고맙습니다. 함께하신 진영 행정안전부 장관님, 이용범 인천시의회 의장님, 박정훈 경찰위원장님, 강영규 경우회장님과 역대 경찰청장님, 그리고 우리 시민 경찰님들을 비롯한 내빈 여러분, 고맙습니다.

감사합니다.

Overwriting data/police.txt


### 1) Dataframe 생성

In [4]:
police = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv(os.path.join('data', 'police.txt'))\
    .withColumnRenamed("_c0", "sentence")

In [5]:
police.printSchema()

root
 |-- sentence: string (nullable = true)



In [6]:
police.show()

+----------------------------------+
|                          sentence|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



### 2) 단어로 분리해서, 출력

In [7]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [8]:
tokDf = tokenizer.transform(police)

In [9]:
tokDf.show()

+----------------------------------+--------------------------------+
|                          sentence|                           words|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|
|         자랑스러운 경찰관 여러분,|   [자랑스러운, 경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
|    대한민국 경찰은 1945년 광복...|    [대한민국, 경찰은, 1945년...|
| 임시정부 초대 경무국장 백범 김...|  [임시정부, 초대, 경무국장, ...|
|    광복 이후 6·25전쟁에서도 경...|    [광복, 이후, 6·25전쟁에서...|
|그러나 잘못도 없지는 않았습니다...|  [그러나, 잘못도, 없지는, 않...|
| 지금 경찰은 과거를 돌아보며 국...|  [지금, 경찰은, 과거를, 돌아...|
| 검경 수사권 조정과 자치경찰제 ...|  [검경, 수사권, 조정과, 자치...|
|정부는 경찰의 근무여건을 개선하...| [정

### 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [20]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

_mystopwords=[u"참",u"큰", u"돌",u"더",u"그",u"될",u"등"]
stop.setStopWords(_mystopwords)

StopWordsRemover_8a5c0669f187

In [21]:
for e in stop.getStopWords():
    print (e, end="/")

참/큰/돌/더/그/될/등/

### 4) 불용어 제거하고, 출력

In [22]:
stopDf=stop.transform(tokDf)
stopDf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                          sentence|                           words|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|    [올해는, ‘경찰의, 날’에, ...|
|         자랑스러운 경찰관 여러분,|   [자랑스러운, 경찰관, 여러분,]|   [자랑스러운, 경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...| [경찰헌장은, “나라

In [23]:
for r in stopDf.select("nostops").take(3):
    for e in r:
        print (e)

['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.']
['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']
['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.']


### 5) TF-IDF를 계산하고, 출력

In [24]:
from pyspark.ml.feature import HashingTF, IDF

In [25]:
hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=50)
hashDf = hashTF.transform(stopDf)

In [26]:
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [27]:
for e in idfDf.select("nostops","idf").take(5):
    print(e)

Row(nostops=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.'], idf=SparseVector(50, {10: 1.0116, 13: 0.904, 17: 0.8938, 23: 0.7885, 36: 2.5986, 48: 0.6931}))
Row(nostops=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.'], idf=SparseVector(50, {0: 1.2993, 1: 0.8938, 3: 5.9264, 5: 0.7885, 8: 1.4816, 10: 1.0116, 12: 1.2993, 14: 0.8938, 16: 1.4816, 17: 0.8938, 18: 0.8938, 19: 0.5261, 25: 1.4816, 26: 0.8938, 28: 0.8938, 29: 0.6931, 33: 3.4095, 37: 1.4816, 48: 0.6931}))
Row(nostops=['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'], idf=SparseVector(50, {1: 0.8938, 2: 1.1451, 3: 2.9632, 5: 0.7885, 6: 2.5986, 7: 1.4816, 11: 1.7047, 14: 0.8938, 15: 1.7876, 19: 0.5261, 21: 1.0116, 24: 1.5769, 26: 0.

### 6) TF-IDF 칼럼을 feature로 구성, 출력

In [21]:
idfDf=idfDf.withColumnRenamed('idf','features')
idfDf.select('features').show(5,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
+-------------------------------------------------------------------------